In [115]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
from PIL import Image
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets, models, transforms
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import splitfolders
from datetime import datetime
import yaml
from sklearn.metrics import classification_report, confusion_matrix

from pipeline_torch import *

In [117]:
config=yaml.load(open('config.yml', 'r'), Loader=yaml.FullLoader)
seed = config['model_config']['initial_seed']
torch.manual_seed(seed)
np.random.seed(seed)

In [34]:
# define training and test data directories
data_dir  = r'..\DL_input_data\ScenarioAlpha\Sensor1'
train_valid_dir = os.path.join(data_dir) 
#test_dir  = os.path.join(data_dir, 'test')

splitfolders.ratio(input=train_valid_dir, output='split_data', ratio=(0.6, 0.4))
train_dir='split_data/train'
valid_dir='split_data/val'

splitfolders.ratio(input='split_data/val', output='Valid_Test', ratio=(0.5, 0.5))
valid_dir='Valid_Test/train'
test_dir='Valid_Test/val'

# Selecting mean and std values according to ImageNet dataset
mean = torch.tensor( [0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])

Copying files: 1160 files [00:11, 99.68 files/s] 
Copying files: 464 files [00:04, 104.04 files/s]


In [118]:
# load and transform data using ImageFolder
data_transforms = {
    'train':  transforms.Compose([
                                transforms.Resize([224,224]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean,std)
                                ]),
    'validation':  transforms.Compose([
                                transforms.Resize([224,224]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean,std)
                                ]),
    'test':  transforms.Compose([
                                transforms.Resize([224,224]),
                                transforms.ToTensor(),
                                transforms.Normalize(mean,std)
                                ])
}

train_data = datasets.ImageFolder(train_dir, transform=data_transforms["train"])
valid_data = datasets.ImageFolder(valid_dir, transform=data_transforms["validation"])
test_data  = datasets.ImageFolder(test_dir, transform=data_transforms["test"])

In [119]:
# Loading the model and freezing the layers
model_transfer = models.resnet50(pretrained=True)
layers=list(model_transfer._modules.keys())

layers_frozen=layers[0:8]

for layer in layers_frozen:
    for param in model_transfer._modules[layer].parameters():
        param.requires_grad=False
        
# modify last layer to match it our classes
n_inputs = model_transfer.fc.in_features
last_layer = nn.Linear(n_inputs, len(train_data.classes))
model_transfer.fc = last_layer

In [120]:
pipeline = PipelineTorch(model_transfer, config)

In [75]:
pipeline.train(train_data, valid_data, config['model_config']['version'])

Epoch [1/5], Learning Rate: 0.000909
Duration of training at epoch 1 is : 0:00:40.299760 seconds.
Epoch: 1 	Training Loss: 1.072934 	Validation Loss: 0.699033
seed has been changed. The new torch seed is 10
Validation loss has descreased (inf-->0.699033). Saving model...
Epoch [2/5], Learning Rate: 0.000800
Duration of training at epoch 2 is : 0:00:41.852769 seconds.
Epoch: 2 	Training Loss: 0.635094 	Validation Loss: 0.545275
Validation loss has descreased (0.699033-->0.545275). Saving model...
Epoch [3/5], Learning Rate: 0.000661
Duration of training at epoch 3 is : 0:00:41.826077 seconds.
Epoch: 3 	Training Loss: 0.432944 	Validation Loss: 0.357440
Validation loss has descreased (0.545275-->0.357440). Saving model...
Epoch [4/5], Learning Rate: 0.000507
Duration of training at epoch 4 is : 0:00:42.096077 seconds.
Epoch: 4 	Training Loss: 0.340277 	Validation Loss: 0.310940
Validation loss has descreased (0.357440-->0.310940). Saving model...
Epoch [5/5], Learning Rate: 0.000352
Dura

In [121]:
pipeline.load_checkpoint(config['model_config']['version'])

=> loading checkpoint 'models\model_v1.pth'
=> loaded checkpoint 'models\model_v1.pth' (epoch 5)


In [122]:
losses, average_loss, predictions, real_labels = pipeline.predict(test_data)    

Test Loss: 0.265883


Test Accuracy: 97% (226/232)


In [126]:
print(classification_report(real_labels, predictions))
print(confusion_matrix(real_labels, predictions))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97        58
           1       1.00      0.97      0.98        58
           2       1.00      0.98      0.99        58
           3       0.93      0.97      0.95        58

    accuracy                           0.97       232
   macro avg       0.97      0.97      0.97       232
weighted avg       0.97      0.97      0.97       232

[[57  0  0  1]
 [ 0 56  0  2]
 [ 0  0 57  1]
 [ 2  0  0 56]]
